In [ ]:
# This file aims to debug the newly created solvent diffusion model with EC drag (add one submodel variable for EC concentration)

In [2]:
import pybamm as pybamm;import pandas as pd   ;import numpy as np;import os;import matplotlib.pyplot as plt;import os;import imageio
from scipy.io import savemat,loadmat;from pybamm import constants,exp;import matplotlib as mpl; fs=17; # or we can set import matplotlib.pyplot as plt then say 'mpl.rc...'
for k in range(0,1):
    mpl.rcParams["axes.labelsize"] = fs
    mpl.rcParams["axes.titlesize"] = fs
    mpl.rcParams["xtick.labelsize"] =  fs
    mpl.rcParams["ytick.labelsize"] =  fs
    mpl.rcParams["legend.fontsize"] =  fs
    mpl.rcParams['font.sans-serif'] = ['Times New Roman']
    mpl.rcParams['font.serif'] = ['Times New Roman']
    mpl.rcParams['axes.unicode_minus'] = False

In [3]:
Model_0  = pybamm.lithium_ion.DFN()
ChemistryChen=pybamm.parameter_sets.Chen2020   # Ren2018  Chen2020_plating
param=pybamm.ParameterValues(chemistry=ChemistryChen);

EC_ratio_Rio   = param.evaluate(Model_0.param.EC_ratio_Rio)
gamma_e_EC_Rio = param.evaluate(Model_0.param.gamma_e_EC_Rio)
tau_discharge  = param.evaluate(Model_0.param.tau_discharge)
tau_EC_Rio     = param.evaluate(Model_0.param.tau_EC_Rio)
Xi             = param.evaluate(Model_0.param.Xi )
Vmolar_Li      = param.evaluate(Model_0.param.Vmolar_Li )
c_ec_0_dim     = param.evaluate(Model_0.param.c_ec_0_dim )
gamma_e        = param.evaluate(Model_0.param.c_ec_0_dim )
Vmolar_EC      = param.evaluate(Model_0.param.Vmolar_EC )
Vmolar_CH2OCO2Li2 = param.evaluate(Model_0.param.Vmolar_CH2OCO2Li2 )
c_e_init_dimensional = param.evaluate(Model_0.param.c_e_init_dimensional )
C_e          = param.evaluate(Model_0.param.C_e )
e_ratio_Rio  = param.evaluate(Model_0.param.e_ratio_Rio )
tau_cross_Rio= param.evaluate(Model_0.param.tau_cross_Rio ) 
D_EC_Li_cross= param.evaluate(Model_0.param.D_EC_Li_cross ) 
print(Vmolar_Li )

1.3e-05


In [ ]:
# variables
tor = variables["Electrolyte tortuosity"]

T = variables["Cell temperature"]
eps_n = variables["Negative electrode porosity"]

sum_s_j = variables["Sum of electrolyte reaction source terms"]
sum_s_j.print_name = "a"
source_terms = sum_s_j /gamma_e


j_inner =  variables["Inner SEI interfacial current density"]
j_outer =  variables["Outer SEI interfacial current density"]
a_n = variables["Negative electrode surface area to volume ratio"]
j_SEI = j_inner + j_outer

In [4]:
x = pybamm.SpatialVariable("x", domain=["negative electrode"], coord_sys="cartesian")
def GetT(x):
    return 25; 
def Getsum_s_j(x):
    return (25)*1e-5;  
def Gettor(x):
    return (25)*0.05;
def Geteps_n(x):
    return 0.37;
def Geta_n(x):
    return 1e5;
def Getj_SEI(x):
    return (25)*5e-5;
def Getdiv_Vbox(x):
    return 0
def Getsign_2_n(x):
    return 1

div_Vbox = Getdiv_Vbox(x)
T = GetT(x)
sum_s_j = Getsum_s_j(x)
tor = Gettor(x)
eps_n = Geteps_n(x)
a_n = Geta_n(x)
j_SEI = Getj_SEI(x)
source_terms = sum_s_j /gamma_e
sign_2_n = Getsign_2_n(x)


In [5]:
# constant definition
D_e = 3e-10; D_EC = 5e-10; t_0plus = 0.3; trans_i = -1.4;  D_cross = -1.5e-9; i_app = 4.8;   F = 9.6E4;
c_init = 1000; c_EC_init= 4000; c_tot_init = 6000; 

In [13]:
model = pybamm.BaseModel()


eps_c_e  = pybamm.Variable("porosity times Li+ Concentration", domain="negative electrode" )
eps_c_EC = pybamm.Variable("porosity times EC Concentration", domain="negative electrode" )

c_e  = eps_c_e /eps_n; # c_e = variables["Electrolyte concentration"]
c_EC = eps_c_EC/eps_n; 


""" model.rhs = {
    eps_c_e: -pybamm.div(-tor * D_e * pybamm.grad(c_e)) / C_e 
            + e_ratio_Rio * gamma_e_EC_Rio * tau_discharge 
            / tau_cross_Rio * pybamm.div(tor * D_EC_Li_cross * pybamm.grad(c_EC))
            + ( 1- t_0plus )  * source_terms 
            -  sign_2_n * (    c_e_init_dimensional * Vmolar_Li   * source_terms         )
            -  (    c_e_init_dimensional / gamma_e * (
                Vmolar_EC-0.5*Vmolar_CH2OCO2Li2) * a_n * j_SEI )
            - c_e * div_Vbox ,
    eps_c_EC:(
                EC_ratio_Rio / gamma_e_EC_Rio * tau_discharge  / 
                tau_cross_Rio * pybamm.div(tor * D_EC_Li_cross * pybamm.grad(c_e))
                )
            + tau_discharge  / tau_EC_Rio * pybamm.div(tor * D_EC * pybamm.grad(c_EC))
            + sign_2_n * (  source_terms / gamma_e / gamma_e_EC_Rio * ( 
                Xi - Vmolar_Li * c_ec_0_dim )   
                )
            +  (  
                a_n * j_SEI /  gamma_e / gamma_e_EC_Rio *   
                ( 1 - Vmolar_EC*c_ec_0_dim - 0.5*Vmolar_CH2OCO2Li2*c_ec_0_dim ) 
                )
    }  """

model.rhs = {
    eps_c_e: -pybamm.div(-tor * D_e * pybamm.grad(c_e)) / C_e 
            + e_ratio_Rio * gamma_e_EC_Rio * tau_discharge 
            / tau_cross_Rio * pybamm.div(tor * D_EC_Li_cross * pybamm.grad(c_EC))
            ,
    eps_c_EC:
            tau_discharge  / tau_EC_Rio * pybamm.div(tor * D_EC * pybamm.grad(c_EC))
    } 

# initial conditions
model.initial_conditions = {eps_c_e: pybamm.Scalar(370),eps_c_EC: pybamm.Scalar(1120)}
# boundary conditions

# calculate for boundary conditions:
a1 = D_e;                               b1 = D_cross * (c_init/c_tot_init);   c1 = -(1-t_0plus)*i_app/F;
a2 = D_cross * (c_EC_init/c_tot_init);  b2 = D_EC;                            c2 = -trans_i*i_app/F;
y_temp = (c2 - a2/a1*c1) / (b2 - a2/a1*b1);   x_temp = (c1 - b1*y_temp) / a1


model.boundary_conditions = {
    c_e: {
        "left": (pybamm.Scalar(1e-3), "Neumann"),"right": (pybamm.Scalar(1e-3), "Neumann")},  
    c_EC: {
        "left": (pybamm.Scalar(1e-3), "Neumann"),"right": (pybamm.Scalar(1e-3), "Neumann")} }
model.variables = {
    "porosity times Li+ Concentration": eps_c_e, 
    "porosity times EC Concentration": eps_c_EC, }
# define geometry
""" x = pybamm.SpatialVariable(
    "x", domain=["negative electrode"], coord_sys="cartesian"
) """
geometry = {"negative electrode": {x: {"min": pybamm.Scalar(0), "max": pybamm.Scalar(8e-3)}}}
# mesh and discretise
submesh_types = {"negative electrode": pybamm.MeshGenerator(pybamm.Uniform1DSubMesh)}
var_pts = {x: 100}
mesh = pybamm.Mesh(geometry, submesh_types, var_pts)
spatial_methods = {"negative electrode": pybamm.FiniteVolume()}
disc = pybamm.Discretisation(mesh, spatial_methods)
disc.process_model(model);

In [ ]:
RHS=model.rhs # 这里有很多个
RHS[list(RHS.keys())[1]].render()



In [14]:
# solve
solver = pybamm.ScipySolver()  #  method='BDF'
t = np.linspace(0, 100, 100)
solution = solver.solve(model, t)

# post-process, so that the solution can be called at any time t or space r
# (using interpolation)
eps_c_e = solution["porosity times Li+ Concentration"].entries;  
eps_c_EC = solution["porosity times EC Concentration"].entries;


2022-07-13 19:11:31,425 - [WARNING] processed_variable.get_spatial_scale(520): No length scale set for negative electrode. Using default of 1 [m].
2022-07-13 19:11:31,437 - [WARNING] processed_variable.get_spatial_scale(520): No length scale set for negative electrode. Using default of 1 [m].


In [15]:
print(eps_c_e)
print(eps_c_EC)

[[  370.          1001.68234704  1264.54559041 ...  2440.21249991
   2420.12739457  2400.16599377]
 [  370.           988.24733851  1254.99861249 ...  2438.16917903
   2418.10276211  2398.16202897]
 [  370.           962.41085547  1236.27475643 ...  2434.08356582
   2414.05571915  2394.15609711]
 ...
 [  370.          -222.40967811  -496.26656976 ... -1694.55893339
  -1674.5346423  -1654.6351847 ]
 [  370.          -248.24188047  -514.9870338  ... -1698.64297397
  -1678.57813631 -1658.64148734]
 [  370.          -261.67680404  -524.53356148 ... -1700.68593841
  -1680.60546459 -1660.64529274]]
[[1120.         1119.99999978 1119.99999968 ... 1119.99999854
  1119.99999854 1119.99999854]
 [1120.         1119.9999998  1119.99999971 ... 1119.99999857
  1119.99999857 1119.99999857]
 [1120.         1119.99999983 1119.99999973 ... 1119.9999986
  1119.9999986  1119.9999986 ]
 ...
 [1120.         1120.00000017 1120.00000027 ... 1120.0000014
  1120.0000014  1120.0000014 ]
 [1120.         1120.0000

In [ ]:
# solve
solver = pybamm.ScipySolver()
t = np.linspace(0, 100, 100)
solution = solver.solve(model, t)

# post-process, so that the solution can be called at any time t or space r
# (using interpolation)
eps_c_e = solution["Li+ Concentration"];  eps_c_EC = solution["EC Concentration"];

# plot
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(13, 5))
x= np.linspace(0, 8e-3, 100)
ax1.plot(x, eps_c_EC(t=0.01, x=x))
ax1.plot(x, eps_c_EC(t=10, x=x))
ax1.plot(x, eps_c_EC(t=60, x=x))
ax1.plot(x, eps_c_EC(t=80, x=x))
ax1.plot(x, eps_c_EC(t=100, x=x))
ax1.set_xlabel("x")
ax1.set_ylabel("Concentration")
ax1.set_title("EC Concentration")


ax2.plot(x, eps_c_e(t=0.01, x=x))
ax2.plot(x, eps_c_e(t=10, x=x))
ax2.plot(x, eps_c_e(t=60, x=x))
ax2.plot(x, eps_c_e(t=80, x=x))
ax2.plot(x, eps_c_e(t=100, x=x))
ax2.set_xlabel("x")
ax2.set_ylabel("Concentration")
ax2.set_title("Li+ Concentration")
plt.tight_layout()
plt.show()

In [ ]:
# Global setting Niall's result:
#Total_Cycles = 1170; Cycle_bt_RPT = 78; Update_Cycles = 78;  Small_Loop =  int(Cycle_bt_RPT/Update_Cycles);   SaveTimes = int(Total_Cycles/Cycle_bt_RPT);   RPT_Cycles = 1; 

# Special cases for Non-dry-out model:
#Total_Cycles_NonDry = 1170-78;   SaveTimes_NonDry = int(Total_Cycles_NonDry/Cycle_bt_RPT);  

#Real_SaveAs_Cycles = 1 # Set this to use it in real experiment so that can really get capacity throughput!
V_max = 4.2;        V_min = 2.5;
cs_Neg_Init = 28920; Diff_SEI = 1.700000000000002e-20;    R_SEI = 2E5;   Bulk_Sol_Con = 4541.0;
Experiment_Long   = pybamm.Experiment( [ (f"Discharge at 1 C until {V_min} V", f"Charge at 0.3 C until {V_max} V", f"Hold at {V_max} V until C/5")    ] * 1 )  
Experiment_RPT    = pybamm.Experiment( [ (f"Discharge at 0.1 C until {V_min} V",  "Rest for 6 hours",  f"Charge at 0.1 C until {V_max} V" ) ] * 1 ) 
Experiment_Breakin= pybamm.Experiment( [ (f"Discharge at 0.2 C until {V_min} V","Rest for 4 hours",f"Charge at 0.2 C until {V_max} V" , f"Hold at {V_max} V until C/100","Rest for 2 hours") *5 + ( f"Discharge at 0.1 C until {V_min} V",  "Rest for 6 hours",  f"Charge at 0.1 C until {V_max} V" ) *1 ] )
#BasicPath = 'D:/OneDrive - Imperial College London/PhD Study/Imperial Study/Model/Pybamm Learning/SimDataSave/SolConsumed/';
#Target = 'Test 20-6 Update=78/';
#if not os.path.exists(BasicPath + Target):
#    os.mkdir(BasicPath + Target);


In [ ]:
# Run model without electrolyte dry out
ChemistryChen=pybamm.parameter_sets.Chen2020  # Ren2018  Chen2020_plating
Para_0=pybamm.ParameterValues(chemistry=ChemistryChen);
Para_0.update({'SEI resistivity [Ohm.m]':R_SEI}) ;
Para_0.update({'Outer SEI solvent diffusivity [m2.s-1]':Diff_SEI});  
Para_0.update({'Bulk solvent concentration [mol.m-3]':Bulk_Sol_Con});
Para_0.update({'Initial concentration in negative electrode [mol.m-3]':cs_Neg_Init }) # 
Sol_NonDry=[];  Model_NonDry=[];
# First Cycle 
Model_0  = pybamm.lithium_ion.DFN() # options={"SEI": "interstitial-diffusion limited","SEI film resistance": "distributed","SEI porosity change": "true" ,"lithium plating": "irreversible","lithium plating porosity change": "true"}
Sim_0    = pybamm.Simulation(Model_0, experiment = Experiment_Long,parameter_values = Para_0,solver = pybamm.CasadiSolver(),)       
Sol_0    = Sim_0.solve();

In [ ]:
Model_0.options

In [ ]:
quick_plot = pybamm.QuickPlot(Sol_0) #     
quick_plot.dynamic_plot();


In [ ]:
"""Check that the model is not under- or over-determined."""
# Equations (differential and algebraic)
# Get all the variables from differential and algebraic equations
vars_in_rhs_keys = set()
vars_in_algebraic_keys = set()
vars_in_eqns = set()
# Get all variables ids from rhs and algebraic keys and equations, and
# from boundary conditions
# For equations we look through the whole expression tree.
# "Variables" can be Concatenations so we also have to look in the whole
# expression tree
unpacker = pybamm.SymbolUnpacker((pybamm.Variable, pybamm.VariableDot))

for var, eqn in Model_0.rhs.items():
    # Find all variables and variabledot objects
    vars_in_rhs_keys_dict = unpacker.unpack_symbol(var)
    vars_in_eqns_dict = unpacker.unpack_symbol(eqn)

    # Store ids only
    # Look only for Variable (not VariableDot) in rhs keys
    vars_in_rhs_keys.update(
        [
            var_id
            for var_id, var in vars_in_rhs_keys_dict.items()
            if isinstance(var, pybamm.Variable)
        ]
    )
    vars_in_eqns.update(vars_in_eqns_dict.keys())
for var, eqn in Model_0.algebraic.items():
    # Find all variables and variabledot objects
    vars_in_algebraic_keys_dict = unpacker.unpack_symbol(var)
    vars_in_eqns_dict = unpacker.unpack_symbol(eqn)

    # Store ids only
    # Look only for Variable (not VariableDot) in algebraic keys
    vars_in_algebraic_keys.update(
        [
            var_id
            for var_id, var in vars_in_algebraic_keys_dict.items()
            if isinstance(var, pybamm.Variable)
        ]
    )
    vars_in_eqns.update(vars_in_eqns_dict.keys())
for var, side_eqn in Model_0.boundary_conditions.items():
    for side, (eqn, typ) in side_eqn.items():
        vars_in_eqns_dict = unpacker.unpack_symbol(eqn)
        vars_in_eqns.update(vars_in_eqns_dict.keys())

# If any keys are repeated between rhs and algebraic then the model is
# overdetermined
if not set(vars_in_rhs_keys).isdisjoint(vars_in_algebraic_keys):
    print("model is overdetermined (repeated keys)")
# If any algebraic keys don't appear in the eqns (or bcs) then the model is
# overdetermined (but rhs keys can be absent from the eqns, e.g. dcdt = -1 is
# fine)
# Skip this step after discretisation, as any variables in the equations will
# have been discretised to slices but keys will still be variables
extra_algebraic_keys = vars_in_algebraic_keys.difference(vars_in_eqns)
# If any variables in the equations don't appear in the keys then the model is
# underdetermined
vars_in_keys = vars_in_rhs_keys.union(vars_in_algebraic_keys)
extra_variables_in_equations = vars_in_eqns.difference(vars_in_keys)

# get ids of external variables
external_ids = {var.id for var in Model_0.external_variables}
for var in Model_0.external_variables:
    if isinstance(var, pybamm.Concatenation):
        child_ids = {child.id for child in var.children}
        external_ids = external_ids.union(child_ids)

extra_variables = extra_variables_in_equations.difference(external_ids)

if extra_variables:
    print("model is underdetermined (too many variables)")


In [ ]:
for var, eqn in Model_0.rhs.items():
    # Find all variables and variabledot objects
    vars_in_rhs_keys_dict = unpacker.unpack_symbol(var)
    vars_in_eqns_dict = unpacker.unpack_symbol(eqn)
    print(vars_in_eqns_dict)
    print(vars_in_rhs_keys_dict)
    print('z')

In [ ]:
for var in Model_0.external_variables:
    print(var)

In [ ]:
print(Model_0.variable_names)
for key in Model_0._variables.keys():
    print(key)

In [ ]:
#print(Model_0.rhs.items())
for var, eqn in Model_0.rhs.items():
    print(var)
    #print(eqn)


In [ ]:
for var, eqn in Model_0.algebraic.items():
    print(var)

In [ ]:
print(extra_variables_in_equations)
print(extra_variables)
print(external_ids)

In [ ]:
Model_0.submodels